# Support Enumeration Algorithm

This algorithm finds all equilibria for any non-degenerate game matrix. It works by trying out every combination of mixed equilibria contatining the same number of rows and columns. The possible equilibria are tested by solving the corresponding system of equations and seeing if the solution gives positive weights to the rows and columns, and seeing if the equilibria contains the "optimal" choices for each player.

It is a generalization of the way we found mixed strategies for 2x2 game matrixes.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import nashpy as nash
import itertools

### The implementation

In [3]:
def supportEnumeration(P1, P2):
    m = P1[:,0].size #number of rows
    n = P1[0,:].size #number of columns
    # Rows - col1, col2, osv.
    for k in range(1, max(m,n) + 1):
        #The possible combinations of rows and colums. i.e. rows 1,3,4 and cols 2,3,6
        combinationsRows = list(itertools.combinations([i for i in range(m)], k))
        combinationsColumns = list(itertools.combinations([i for i in range(n)], k))
        for combiCol in combinationsColumns: #Iterates over combinations of columns
            for combiRow in combinationsRows: #Iterates over combinations of rows
                
                #-----Finding the weights the rows must have to make the selected columns of equal fitness to player 2----------
                cols = [P2[j] for j in combiCol] #The columns we are making P2 choose between
                A = []
                for i in combiRow:
                    eq = [cols[j][i] for j in range(k)] #Equation finding the weights that makes these columns the same fitness
                    eq = eq + [-1] #Adding -1 at the end of the equation to represent the fitness
                    A.append(eq) #Adding the equation to our set of equations
                A.append([1 for i in range(k)] + [0]) #The final equation makes all the weights add to 1
                A = np.array(A)
                b = np.array([0 for i in range(k)] + [1]) #The right side of our equations. They are all 0, except for the sum of the weights which is 1
                try:
                    rowSolution = np.linalg.solve(A, b)
                except np.linalg.LinAlgError:
                    continue #Skips to next rows/colums combination if there is not solution
                
                colPayoff = rowSolution[-1]
                rowWeight = [0] * n
                i = 0
                for j in combiCol:
                    rowWeight[j] = rowSolution[i]
                    i += 1
    
                #-----Finding the weights the columns must have to make the selected rows of equal fitness to player 1----------
                rows = [P1[:,j] for j in combiRow]
                C = []
                for i in combiCol:
                    eq = [rows[j][i] for j in range(k)]
                    eq = eq + [-1]
                    C.append(eq)
                C.append([1 for i in range(k)] + [0])
                C = np.array(C)
                d = np.array([0 for i in range(k)] + [1])
                try:
                    colSolution = np.linalg.solve(C, d)
                except np.linalg.LinAlgError:
                    continue
                
                rowPayoff = colSolution[-1]
                colWeight = [0] * n
                i = 0
                for j in combiRow:
                    colWeight[j] = colSolution[i]
                    i += 1
                #------The solution is not valid if the weights are negative or if a non-selected strategy has a higher payoff-------
                valid = True
                for weight in rowWeight:
                    if weight + 1e-9 < 0:
                        valid = False
                for weight in colWeight:
                    if weight + 1e-9 < 0:
                        valid = False
                for payoff in rowWeight @ P2:
                    if payoff > colPayoff + 1e-9:
                        valid = False
                for payoff in P1 @ colWeight:
                    if payoff > rowPayoff + 1e-9:
                        valid = False
                
                if valid:
                    print(f"P1:{rowWeight} - {rowPayoff}\nP2:{colWeight} - {colPayoff}\n")

### Testing

In [4]:
P1 = np.array([[2, 6, 5],
               [4, -1, -3],
               [2, 2, 5]])

P2 = np.array([[2, 4, 2],
               [0, -1, 2],
               [5, -3, 2]])

In [5]:
supportEnumeration(P1, P2)

P1:[1.0, 0, 0] - 6.0
P2:[0, 1.0, 0] - 4.0

P1:[0, 0.6, 0.4] - 2.6
P2:[0.8, 0, 0.2] - 2.0

P1:[0.6551724137931034, 0.20689655172413796, 0.13793103448275862] - 2.6
P2:[0.8, -4.270088556250602e-18, 0.2] - 2.0



We can compare with the implementation of support enumeration and vertex enumeration from nashpy. Surprisingly, nashpy fails to find all equilibria with its implementation of support enumeration.

In [6]:
rps = nash.Game(P1, P2)
print(rps)
eqs = rps.support_enumeration()
for equ in eqs:
    print(equ)

Bi matrix game with payoff matrices:

Row player:
[[ 2  6  5]
 [ 4 -1 -3]
 [ 2  2  5]]

Column player:
[[ 2  4  2]
 [ 0 -1  2]
 [ 5 -3  2]]
(array([1., 0., 0.]), array([0., 1., 0.]))


In [7]:
eqs = rps.vertex_enumeration()
for equ in eqs:
    print(equ)

(array([3.46944695e-17, 6.00000000e-01, 4.00000000e-01]), array([0.8, 0. , 0.2]))
(array([ 1.00000000e+00, -4.85722573e-17,  4.85722573e-17]), array([ 0.00000000e+00,  1.00000000e+00, -3.12250226e-17]))
(array([0.65517241, 0.20689655, 0.13793103]), array([0.8, 0. , 0.2]))


In [8]:
#Stag hunt - p.49
a1 = 5; b1 = 0; c1 = 4; d1 = 2
a2 = 5; b2 = 4; c2 = 0; d2 = 2

P1 = np.array([[a1, b1],
               [c1, d1]])

P2 = np.array([[a2, b2],
               [c2, d2]])

supportEnumeration(P1, P2)

P1:[1.0, 0] - 5.0
P2:[1.0, 0] - 5.0

P1:[0, 1.0] - 2.0
P2:[0, 1.0] - 2.0

P1:[0.6666666666666667, 0.33333333333333326] - 3.3333333333333335
P2:[0.6666666666666667, 0.33333333333333326] - 3.3333333333333335

